In [1]:
from config import Config
import googlemaps

conf = Config()
gClient = googlemaps.Client(key=conf.PLACES_API_KEY)



 API queries_quota: 60 



In [2]:
from time import sleep

I_location = (47.47254804521419, 19.059588492595783)
result = gClient.places_nearby(location=I_location,radius=1000,type='bar')
bars = result['results']
#limited  do while cicle
for _ in range(10):
    #check for paging token
    if 'next_page_token' not in result:
        break
    # credit to https://stackoverflow.com/a/15452661/8659684
    # the docs say, that there is a delay in pages getting public, I guess it is throttling
    sleep(2)
    print('.',end='')
    result = gClient.places_nearby(location=I_location,radius=1000,type='bar',page_token=result['next_page_token'])
    bars += result['results']
print(f' ({len(bars)})')

.

KeyboardInterrupt: 

In [ ]:
keys = [k for k in bars[0]]
print(keys,len(bars))

['business_status', 'geometry', 'icon', 'icon_background_color', 'icon_mask_base_uri', 'name', 'opening_hours', 'photos', 'place_id', 'plus_code', 'price_level', 'rating', 'reference', 'scope', 'types', 'user_ratings_total', 'vicinity'] 43


In [ ]:
import haversine as hs
import pandas as pd

#print(f"{bar['name']} dist: {hs.haversine(I_location,loc,unit=hs.Unit.METERS):4.5}")

df = pd.DataFrame(bars)

In [ ]:
df['location'] = [(g['location']['lat'],g['location']['lng']) for g in df['geometry']]
df['distance'] = [hs.haversine(I_location,g,unit=hs.Unit.METERS) for g in df['location']]
df2 = df.sort_values('distance')
print(df2[['name','place_id','distance']])

                                        name                     place_id  \
14                            Eufória terasz  ChIJp5T3ygvdQUcRaWdcRcxDDp0   
15                                   Szertár  ChIJnb4U3qvdQUcRxxCi905Ugxc   
13                              Kamra Campus  ChIJ7QqCQqzdQUcRcfyFqiSFdJ0   
16                                    Spaten  ChIJvUMTaJPdQUcR1qAiVED_4Wg   
17                               Karman Club  ChIJBbzOo6zdQUcRJqUVsYRTHLA   
0                            Napközi Bisztró  ChIJu2fffK7dQUcRgbC2JH1_I24   
18                    Eleven Bistro Budapest  ChIJW2flWgTdQUcR48i7ulRVjbg   
19                             Duna Bár Kft.  ChIJawp1OwbdQUcRceXXj0oDeRA   
1                                 Pinyó Klub  ChIJZTGpDa3dQUcR1GFtzxxGDak   
20                                Nemaza Bár  ChIJDfwjuxfdQUcRh0tk5yohBnI   
21                         Nu Buda Jazz Club  ChIJkUBtli3dQUcRuV6y871em2w   
22                        Martos Csocsó Klub  ChIJJS9-2FLcQUcRtmAqCfspgX0   

In [ ]:
karman_id = 'ChIJBbzOo6zdQUcRJqUVsYRTHLA'

place_result = gClient.place(place_id=karman_id)

This can be used for opening hour determination, but it will be a bit more challenging, than the distance calculation.


In [ ]:

print(place_result['result']['current_opening_hours']['weekday_text'])
for opclotimer in place_result['result']['current_opening_hours']['periods']:
    print(opclotimer)

['Monday: 11:00 AM – 1:00 AM', 'Tuesday: 11:00 AM – 1:00 AM', 'Wednesday: 11:00 AM – 12:00 AM', 'Thursday: 11:00 AM – 2:00 AM', 'Friday: 11:00 AM – 2:00 AM', 'Saturday: Closed', 'Sunday: Closed']
{'close': {'date': '2022-11-28', 'day': 1, 'time': '2359', 'truncated': True}, 'open': {'date': '2022-11-28', 'day': 1, 'time': '1100'}}
{'close': {'date': '2022-11-22', 'day': 2, 'time': '0100'}, 'open': {'date': '2022-11-22', 'day': 2, 'time': '0000', 'truncated': True}}
{'close': {'date': '2022-11-23', 'day': 3, 'time': '0100'}, 'open': {'date': '2022-11-22', 'day': 2, 'time': '1100'}}
{'close': {'date': '2022-11-24', 'day': 4, 'time': '0000'}, 'open': {'date': '2022-11-23', 'day': 3, 'time': '1100'}}
{'close': {'date': '2022-11-25', 'day': 5, 'time': '0200'}, 'open': {'date': '2022-11-24', 'day': 4, 'time': '1100'}}
{'close': {'date': '2022-11-26', 'day': 6, 'time': '0200'}, 'open': {'date': '2022-11-25', 'day': 5, 'time': '1100'}}


In [29]:
import src.Places as plc
from importlib import reload
import pandas as pd

reload(plc)
df = pd.DataFrame({'stop_lat':[47.47254804521419,47.47254804521419],'stop_lon':[19.059588492595783,19.059588492595783],'radius':[300,2000]})

client = plc.Places()


 API queries_quota: 60 



In [23]:
res = client.get_bars_batch(df)

100%|██████████| 2/2 [00:08<00:00,  4.44s/it]


In [24]:
res.head()

,business_status,geometry,name,place_id,plus_code,price_level,rating,types,user_ratings_total,vicinity,permanently_closed,bar_lat,bar_lon
0,OPERATIONAL,"{'location': {'lat': 47.4731179, 'lng': 19.051...",Napközi Bisztró,ChIJu2fffK7dQUcRgbC2JH1_I24,"{'compound_code': 'F3F2+6M Budapest, Hungary',...",2.0,3.8,"[bar, point_of_interest, establishment]",470.0,"Budapest 35, Október huszonharmadika utca 33",NaN,47.473118,19.051701
1,OPERATIONAL,"{'location': {'lat': 47.4766196, 'lng': 19.052...",Pinyó Klub,ChIJZTGpDa3dQUcR1GFtzxxGDak,"{'compound_code': 'F3G3+J3 Budapest, Hungary',...",2.0,4.2,"[bar, point_of_interest, establishment]",897.0,"Budapest, Karinthy Frigyes út 26",NaN,47.476620,19.052718
2,OPERATIONAL,"{'location': {'lat': 47.4778761, 'lng': 19.051...",Black Dog Pub,ChIJU2nbW63dQUcRiUtmk5Vn6Fw,"{'compound_code': 'F3H2+5P Budapest, Hungary',...",2.0,4.5,"[bar, point_of_interest, establishment]",687.0,"Budapest, Lágymányosi utca 12",NaN,47.477876,19.051786
3,OPERATIONAL,"{'location': {'lat': 47.4750225, 'lng': 19.048...",Fruitisimo,ChIJjxoxCK7dQUcRPO000x4k6t0,"{'compound_code': 'F2GX+2H Budapest, Hungary',...",NaN,3.7,"[bar, store, food, point_of_interest, health, ...",57.0,"Budapest, Október huszonharmadika utca 8-10",NaN,47.475023,19.048976
4,OPERATIONAL,"{'location': {'lat': 47.4775751, 'lng': 19.051...",Happy Drink Bar,ChIJcbrFobLdQUcRYLkRKfyx0iU,"{'compound_code': 'F3H2+2C Budapest, Hungary',...",1.0,4.7,"[bar, point_of_interest, establishment]",527.0,"Budapest, Bercsényi utca 12",NaN,47.477575,19.051113


In [27]:
print(res['permanently_closed'].count())
print(res.size)

16
1170


In [30]:
res2 = client.get_bars_ranged_batch(df,'radius')

100%|██████████| 2/2 [00:04<00:00,  2.44s/it]


In [31]:
print(res2['permanently_closed'].count())
print(res2.size)

7
806
